In [1]:
import requests
from bs4 import BeautifulSoup
import sqlite3

In [2]:
url = "http://books.toscrape.com/catalogue/category/books/fiction_10/index.html"
response = requests.request('GET',url)
#print(response.text)
soup = BeautifulSoup(response.text,'html.parser')

In [3]:
books = []
for x in soup.find_all('a'):
    if x.get('title') != None:
        books.append(x.get('title'))
print(books)

['Soumission', 'Private Paris (Private #10)', 'We Love You, Charlie Freeman', 'Thirst', 'The Murder That Never Was (Forensic Instincts #5)', 'Tuesday Nights in 1980', 'The Vacationers', 'The Regional Office Is Under Attack!', 'Finders Keepers (Bill Hodges Trilogy #2)', 'The Time Keeper', 'The Testament of Mary', 'The First Hostage (J.B. Collins #2)', 'Take Me with You', 'Still Life with Bread Crumbs', 'Shtum', 'My Name Is Lucy Barton', 'My Mrs. Brown', 'Mr. Mercedes (Bill Hodges Trilogy #1)', 'I Am Pilgrim (Pilgrim #1)', 'Eligible (The Austen Project #4)']


In [4]:
prices = []
price = soup.find_all('p',attrs={'class':'price_color'})
for x in range(len(price)):
    prices.append(float(price[x].text[2:]))
print(prices)

[50.1, 47.61, 50.27, 17.27, 54.11, 21.04, 42.15, 51.36, 53.53, 27.88, 52.67, 25.85, 45.21, 26.41, 55.84, 41.56, 24.48, 28.9, 10.6, 27.09]


In [5]:
rating = []
availability = []
for x in soup.find_all('p'):
    if len(x.get('class')) > 1:
        if x.get('class')[0] == 'star-rating':
            rating.append(x.get('class')[1])
        else:
            availability.append(x.get('class')[0])
print(rating)
print(availability)

['One', 'Five', 'Five', 'Five', 'Three', 'Two', 'Four', 'Five', 'Five', 'Five', 'Four', 'Three', 'Three', 'Three', 'Four', 'One', 'Three', 'One', 'Four', 'Three']
['instock', 'instock', 'instock', 'instock', 'instock', 'instock', 'instock', 'instock', 'instock', 'instock', 'instock', 'instock', 'instock', 'instock', 'instock', 'instock', 'instock', 'instock', 'instock', 'instock']


In [6]:
y = " ".join(rating)
z = y.replace('One','1').replace('Two','2').replace('Three','3').replace('Four','4').replace('Five','5').split(" ")
rating_int = []
for x in z:
    rating_int.append(int(x))
rating_int

[1, 5, 5, 5, 3, 2, 4, 5, 5, 5, 4, 3, 3, 3, 4, 1, 3, 1, 4, 3]

In [7]:
connection = sqlite3.connect("book_store.db")
cursor = connection.cursor()
for x in range(len(books)):
    cursor.execute("INSERT INTO books (book,price,rate,availability) VALUES (?,?,?,?)",\
                   [books[x],prices[x],rating_int[x],availability[x]])
    connection.commit()
connection.close()

In [8]:
connection = sqlite3.connect("book_store.db")
cursor = connection.cursor()
cursor.execute("SELECT * FROM books")
rows = cursor.fetchall()
connection.close()

for Id,book,price,rate,availability in rows:
    print(f"ID: {Id}, Book: {book}, Price: {price}, Rate: {rate}, Availability: {availability}")

ID: 1, Book: Soumission, Price: 50.1, Rate: 1, Availability: instock
ID: 2, Book: Private Paris (Private #10), Price: 47.61, Rate: 5, Availability: instock
ID: 3, Book: We Love You, Charlie Freeman, Price: 50.27, Rate: 5, Availability: instock
ID: 4, Book: Thirst, Price: 17.27, Rate: 5, Availability: instock
ID: 5, Book: The Murder That Never Was (Forensic Instincts #5), Price: 54.11, Rate: 3, Availability: instock
ID: 6, Book: Tuesday Nights in 1980, Price: 21.04, Rate: 2, Availability: instock
ID: 7, Book: The Vacationers, Price: 42.15, Rate: 4, Availability: instock
ID: 8, Book: The Regional Office Is Under Attack!, Price: 51.36, Rate: 5, Availability: instock
ID: 9, Book: Finders Keepers (Bill Hodges Trilogy #2), Price: 53.53, Rate: 5, Availability: instock
ID: 10, Book: The Time Keeper, Price: 27.88, Rate: 5, Availability: instock
ID: 11, Book: The Testament of Mary, Price: 52.67, Rate: 4, Availability: instock
ID: 12, Book: The First Hostage (J.B. Collins #2), Price: 25.85, Rate:

In [9]:
connection = sqlite3.connect("book_store.db")
cursor = connection.cursor()
cursor.execute("SELECT id,book,rate FROM books WHERE rate >= 3 ORDER BY rate")
rows = cursor.fetchall()
connection.close()

for Id,book,rate in rows:
    print(f"ID: {Id}, Book: {book}, Rate: {rate}")

ID: 5, Book: The Murder That Never Was (Forensic Instincts #5), Rate: 3
ID: 12, Book: The First Hostage (J.B. Collins #2), Rate: 3
ID: 13, Book: Take Me with You, Rate: 3
ID: 14, Book: Still Life with Bread Crumbs, Rate: 3
ID: 17, Book: My Mrs. Brown, Rate: 3
ID: 20, Book: Eligible (The Austen Project #4), Rate: 3
ID: 7, Book: The Vacationers, Rate: 4
ID: 11, Book: The Testament of Mary, Rate: 4
ID: 15, Book: Shtum, Rate: 4
ID: 19, Book: I Am Pilgrim (Pilgrim #1), Rate: 4
ID: 2, Book: Private Paris (Private #10), Rate: 5
ID: 3, Book: We Love You, Charlie Freeman, Rate: 5
ID: 4, Book: Thirst, Rate: 5
ID: 8, Book: The Regional Office Is Under Attack!, Rate: 5
ID: 9, Book: Finders Keepers (Bill Hodges Trilogy #2), Rate: 5
ID: 10, Book: The Time Keeper, Rate: 5


In [10]:
connection = sqlite3.connect("book_store.db")
cursor = connection.cursor()
cursor.execute("SELECT id,book,price FROM books ORDER BY price DESC LIMIT 5")
rows = cursor.fetchall()
connection.close()

for Id,book,price in rows:
    print(f"ID: {Id}, Book: {book}, Price: {price}")

ID: 15, Book: Shtum, Price: 55.84
ID: 5, Book: The Murder That Never Was (Forensic Instincts #5), Price: 54.11
ID: 9, Book: Finders Keepers (Bill Hodges Trilogy #2), Price: 53.53
ID: 11, Book: The Testament of Mary, Price: 52.67
ID: 8, Book: The Regional Office Is Under Attack!, Price: 51.36


In [11]:
connection = sqlite3.connect("book_store.db")
cursor = connection.cursor()
cursor.execute("SELECT id,book FROM books WHERE book like '%Mr%'")
rows = cursor.fetchall()
connection.close()

for Id,book in rows:
    print(f"ID: {Id}, Book: {book}")

ID: 18, Book: Mr. Mercedes (Bill Hodges Trilogy #1)
ID: 17, Book: My Mrs. Brown
